In [6]:
from configs.config import prompt_config
import requests
from bs4 import BeautifulSoup
from utils.util import *
import time
import random
import requests
from bs4 import BeautifulSoup
from langchain_openai import ChatOpenAI
from playwright.sync_api import sync_playwright
from playwright.async_api import async_playwright
from datetime import datetime, timedelta
import json
import asyncio

In [7]:
llm = ChatOpenAI(model="gpt-4o")

In [8]:
def parse_relative_date(relative_date: str) -> str:
    """
        Des:
            날짜 표현 변환 함수
        Args:
            relative_date (str): 상대적인 날짜 표현 (~시간 전, ~일 전 표현)
        Returns:
            str: 절대 날짜 (YYYY. MM. DD.)
    """
    now = datetime.now()

    if "시간 전" in relative_date:
        hours = int(re.search(r"(\d+)", relative_date).group(1))
        result_date = now - timedelta(hours=hours)
    elif "일 전" in relative_date:
        days = int(re.search(r"(\d+)", relative_date).group(1))
        result_date = now - timedelta(days=days)
    elif "분 전" in relative_date:
        minutes = int(re.search(r"(\d+)", relative_date).group(1))
        result_date = now - timedelta(minutes=minutes)
    else:
        return relative_date

    return result_date.strftime("%Y. %m. %d.")

In [18]:
async def search_google(search_term):
    async with async_playwright() as p:
        # 브라우저 설정 수정
        browser = await p.chromium.launch(
            headless=True,  # headless 모드를 끄고
            args=[
                '--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
                '--disable-blink-features=AutomationControlled',
                '--no-sandbox',
                '--disable-setuid-sandbox',
            ]
        )
        
        # 브라우저 컨텍스트에 추가 설정
        context = await browser.new_context(
            viewport={'width': 1920, 'height': 1080},
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            java_script_enabled=True,
        )
        
        page = await context.new_page()
        
        # 자동화 감지 우회를 위한 추가 설정
        await page.add_init_script("""
            Object.defineProperty(navigator, 'webdriver', {
                get: () => false,
            });
        """)
        
        try:
            # 랜덤한 지연 시간 추가
            await page.wait_for_timeout(2000 + random.randint(500, 1500))
            
            await page.goto('https://www.google.com')
            await page.wait_for_timeout(1000 + random.randint(500, 1000))
            
            # 검색어 입력을 더 자연스럽게
            await page.type('#APjFqb', search_term, delay=100)
            await page.wait_for_timeout(500)
            await page.press('#APjFqb', 'Enter')
            
            # 검색 결과 대기
            await page.wait_for_selector('div.yuRUbf', timeout=10000)
            
            # 결과 추출 전 잠시 대기
            await page.wait_for_timeout(2000)
            
            results = []
            search_results = await page.query_selector_all('div.yuRUbf')
            
            for result in search_results:
                title_elem = await result.query_selector('h3.LC20lb')
                title = await title_elem.inner_text() if title_elem else ''
                
                link_elem = await result.query_selector('a')
                link = await link_elem.get_attribute('href') if link_elem else ''
                
                results.append({
                    'title': title,
                    'link': link
                })
            
            return results
            
        except Exception as e:
            print(f"에러 발생: {str(e)}")
            await page.screenshot(path='error.png')
            raise e
            
        finally:
            await browser.close()

In [19]:
# import subprocess
# subprocess.run(['playwright', 'install', 'chromium'])

In [20]:
from tqdm import tqdm
for _ in tqdm(range(100)):
    search_keyword = "2025년 1월 중순 배추 가격 동향"
    results = await search_google(search_keyword)
    print("\n제목:", results[0]['title'])
    print("링크:", results[0]['link'])

  1%|          | 1/100 [00:11<19:15, 11.67s/it]


제목: 소매가격 > 소비정보 - 농산물유통 종합 ...
링크: https://www.nongnet.or.kr/front/M000000262/somaeInfo/somae.do?searchDate=2025%EB%85%84%2001%EC%9B%94%2007%EC%9D%BC&searchSymbol1=211&searchName1=%EB%B0%B0%EC%B6%94&searchSymbol2=&searchName2=


  2%|▏         | 2/100 [00:24<19:43, 12.08s/it]


제목: 소매가격 > 소비정보 - 농산물유통 종합 ...
링크: https://www.nongnet.or.kr/front/M000000262/somaeInfo/somae.do?searchDate=2025%EB%85%84%2001%EC%9B%94%2007%EC%9D%BC&searchSymbol1=211&searchName1=%EB%B0%B0%EC%B6%94&searchSymbol2=&searchName2=


  3%|▎         | 3/100 [00:35<19:15, 11.91s/it]

...


제목: 소매가격 > 소비정보 - 농산물유통 종합 ...
링크: https://www.nongnet.or.kr/front/M000000262/somaeInfo/somae.do?searchDate=2025%EB%85%84%2001%EC%9B%94%2007%EC%9D%BC&searchSymbol1=211&searchName1=%EB%B0%B0%EC%B6%94&searchSymbol2=&searchName2=


 49%|████▉     | 49/100 [09:44<10:02, 11.81s/it]


제목: 소매가격 > 소비정보 - 농산물유통 종합 ...
링크: https://www.nongnet.or.kr/front/M000000262/somaeInfo/somae.do?searchDate=2025%EB%85%84%2001%EC%9B%94%2007%EC%9D%BC&searchSymbol1=211&searchName1=%EB%B0%B0%EC%B6%94&searchSymbol2=&searchName2=


 50%|█████     | 50/100 [09:55<09:40, 11.61s/it]


제목: 소매가격 > 소비정보 - 농산물유통 종합 ...
링크: https://www.nongnet.or.kr/front/M000000262/somaeInfo/somae.do?searchDate=2025%EB%85%84%2001%EC%9B%94%2007%EC%9D%BC&searchSymbol1=211&searchName1=%EB%B0%B0%EC%B6%94&searchSymbol2=&searchName2=


In [4]:
import requests

url = "https://www.google.com/search?q=2025%EB%85%84+1%EC%9B%94+%EC%A4%91%EC%88%9C+%EB%B0%B0%EC%B6%94+%EA%B0%80%EA%B2%A9+%EB%8F%99%ED%96%A5&sca_esv=3baac282e3bcd052&sxsrf=ADLYWIIz36jxuFM5uAuo3cIi8FohhZ4xMw%3A1737176495838&source=hp&ei=rzWLZ5zuMO7q1e8P_pu26AI&iflsig=AL9hbdgAAAAAZ4tDv1RmFbuBI1aeVjl1-RRws2dgub69&ved=0ahUKEwjcgcffvv6KAxVudfUHHf6NDS0Q4dUDCBk&uact=5&oq=2025%EB%85%84+1%EC%9B%94+%EC%A4%91%EC%88%9C+%EB%B0%B0%EC%B6%94+%EA%B0%80%EA%B2%A9+%EB%8F%99%ED%96%A5&gs_lp=Egdnd3Mtd2l6IigyMDI164WEIDHsm5Qg7KSR7IicIOuwsOy2lCDqsIDqsqkg64-Z7ZalMgQQIxgnMggQABiABBiiBDIIEAAYgAQYogQyCBAAGIAEGKIEMggQABiABBiiBEi3AlCVAViVAXABeACQAQCYAZMBoAGTAaoBAzAuMbgBA8gBAPgBAvgBAZgCAqACnwGoAgrCAgcQIxgnGOoCmAMK8QX_E4DQkuKZIpIHAzEuMaAH8gQ&sclient=gws-wiz"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
response = requests.get(url, headers=headers)

In [5]:
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
soup

<!DOCTYPE html>
<html lang="ko"><head><title>Google Search</title><style>body{background-color:var(--xhUGwc)}</style></head><body><noscript><style>table,div,span,p{display:none}</style><meta content="0;url=/httpservice/retry/enablejs?sei=izmLZ6rTAsbf1e8PzKvLiQQ" http-equiv="refresh"/><div style="display:block">몇 초 안에 이동하지 않는 경우 <a href="/httpservice/retry/enablejs?sei=izmLZ6rTAsbf1e8PzKvLiQQ">여기</a>를 클릭하세요.</div></noscript><script nonce="Vj7mnMI8caSgIoDeOEC72w">//# sourceMappingURL=data:application/json;charset=utf-8;base64,eyJ2ZXJzaW9uIjogMywic291cmNlcyI6WyIiXSwic291cmNlc0NvbnRlbnQiOlsiICJdLCJuYW1lcyI6WyJjbG9zdXJlRHluYW1pY0J1dHRvbiJdLCJtYXBwaW5ncyI6IkFBQUE7QUFBQTtBQUFBO0FBQUE7QUFBQTtBQUFBO0FBQUEifQ==
(function(){function p(b){return b}var M=function(b){return p.call(this,b)},Z=function(b,G,L,d,I,H,W,C,z,r,g,a){for(g=(a=60,73);;)try{if(a==6)break;else if(a==d)a=w.console?G:32;else{if(a==32)return g=73,z;if(a==69)a=C&&C.createPolicy?64:b;else if(a==60)z=W,C=w.trustedTypes,a=69;else if(a

In [ ]:
search_keyword = "2025년 1월 중순 배추 가격 동향"
results = google_search_scrape(search_keyword, SEARCH_RESULT_COUNT=6)
print(f"{RED}검색어 : {search_keyword}\n검색결과 : {len(results)}\n{RESET}")
main_context = ''
suffix_context = ''
for idx, result in enumerate(results):
    link = result.get("link")
    date = result.get("date")
    try:
        desc, detailed_content = extract_content(link)
    except:
        pass
    try:
        if "Enable JavaScript and cookies" in detailed_content:
            continue
    except:
        continue
    main_context += f"제목 : {result.get('title')}\n링크 : {link}\n설명 : {desc}\n내용 : {detailed_content}\n\n"    
    suffix_context += f"""
📌 참고내용 [{idx+1}]
제목 : {result.get('title')}
링크 : {link}
설명 : {desc}
날짜 : {date}
"""